In [1]:
# imports 
import numpy as np
from math import sqrt 
from pprint import pprint
from numpy import array
from scipy.io import arff
import pandas as pd
from time import time

In [2]:
# load data
df=pd.read_csv('PenDigits_withoutdupl_norm_v01.csv')  

del df['id']
del df['Unnamed: 0']
df['outlier'] = df.outlier.apply(lambda label: 1 if label == "'yes'" else 0)

df.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,att11,att12,att13,att14,att15,att16,outlier
0,48.0,100.0,20.0,80.0,0.0,51.0,55.0,43.0,100.0,56.0,90.0,62.0,86.0,30.0,73.0,0.0,1
1,38.0,100.0,4.0,85.0,0.0,53.0,58.0,46.0,100.0,52.0,88.0,64.0,78.0,32.0,73.0,0.0,1
2,53.0,100.0,25.0,70.0,0.0,39.0,50.0,34.0,100.0,42.0,85.0,64.0,77.0,32.0,80.0,0.0,1
3,12.0,100.0,0.0,71.0,25.0,43.0,86.0,44.0,100.0,75.0,90.0,52.0,90.0,20.0,90.0,0.0,1
4,26.0,100.0,0.0,83.0,23.0,51.0,83.0,42.0,100.0,55.0,79.0,84.0,56.0,42.0,37.0,0.0,1


In [3]:
df.shape

(9868, 17)

In [4]:
# data to values
df = df.values

In [5]:
# Extratrees Deep
import argparse
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/Pendigits.json")
gc = GCForest(config)

In [6]:
# train test 
from sklearn.cross_validation import train_test_split
y = df[:,16]
X = df[:,0:16]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
# number of class
len(np.unique(y_train))

2

In [8]:
           # X_enc is the concatenated predict_proba result of ExtraTrees

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
   

[ 2018-04-26 15:03:09,851][cascade_classifier.fit_transform] X_groups_train.shape=[(6611, 16)],y_train.shape=(6611,),X_groups_test.shape=[(3257, 16)],y_test.shape=(3257,)
[ 2018-04-26 15:03:09,853][cascade_classifier.fit_transform] group_dims=[16]
[ 2018-04-26 15:03:09,855][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-26 15:03:09,856][cascade_classifier.fit_transform] group_ends=[16]
[ 2018-04-26 15:03:09,857][cascade_classifier.fit_transform] X_train.shape=(6611, 16),X_test.shape=(3257, 16)
[ 2018-04-26 15:03:09,859][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(6611, 16), X_cur_test.shape=(3257, 16)
[ 2018-04-26 15:03:10,658][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=100.00%
[ 2018-04-26 15:03:11,735][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.85%
[ 2018-04-26 15:03:12,847][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 

In [9]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-26 15:03:20,969][cascade_classifier.transform] X_groups_test.shape=[(3257, 16)]
[ 2018-04-26 15:03:20,972][cascade_classifier.transform] group_dims=[16]
[ 2018-04-26 15:03:20,974][cascade_classifier.transform] X_test.shape=(3257, 16)
[ 2018-04-26 15:03:20,976][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(3257, 16)


Test Accuracy of ExtraTrees = 99.877188 %


In [10]:
# Make data + estimators
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-04-26 15:03:23,505][cascade_classifier.transform] X_groups_test.shape=[(3257, 16)]
[ 2018-04-26 15:03:23,508][cascade_classifier.transform] group_dims=[16]
[ 2018-04-26 15:03:23,509][cascade_classifier.transform] X_test.shape=(3257, 16)
[ 2018-04-26 15:03:23,511][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(3257, 16)


In [11]:
X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-04-26 15:03:25,964][cascade_classifier.fit_transform] X_groups_train.shape=[(6611, 18)],y_train.shape=(6611,),X_groups_test.shape=[(3257, 18)],y_test.shape=(3257,)
[ 2018-04-26 15:03:25,967][cascade_classifier.fit_transform] group_dims=[18]
[ 2018-04-26 15:03:25,970][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-26 15:03:25,972][cascade_classifier.fit_transform] group_ends=[18]
[ 2018-04-26 15:03:25,973][cascade_classifier.fit_transform] X_train.shape=(6611, 18),X_test.shape=(3257, 18)
[ 2018-04-26 15:03:25,975][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(6611, 18), X_cur_test.shape=(3257, 18)
[ 2018-04-26 15:03:26,794][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=100.00%
[ 2018-04-26 15:03:27,753][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=100.00%
[ 2018-04-26 15:03:28,662][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0

In [12]:
# predict  data + Es
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-26 15:03:36,823][cascade_classifier.transform] X_groups_test.shape=[(3257, 18)]
[ 2018-04-26 15:03:36,827][cascade_classifier.transform] group_dims=[18]
[ 2018-04-26 15:03:36,828][cascade_classifier.transform] X_test.shape=(3257, 18)
[ 2018-04-26 15:03:36,831][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(3257, 18)


Test Accuracy of ExtraTrees = 99.969297 %


In [13]:
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100),i,'fois')

In [14]:
# Deep
for i in range (20):
    t0 = time()
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    tt = time() - t0
    affiche() 
    print (" Time ",format(round(tt,3))," seconds")

[ 2018-04-26 15:03:39,458][cascade_classifier.fit_transform] X_groups_train.shape=[(6611, 18)],y_train.shape=(6611,),X_groups_test.shape=[(3257, 18)],y_test.shape=(3257,)
[ 2018-04-26 15:03:39,460][cascade_classifier.fit_transform] group_dims=[18]
[ 2018-04-26 15:03:39,461][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-26 15:03:39,462][cascade_classifier.fit_transform] group_ends=[18]
[ 2018-04-26 15:03:39,464][cascade_classifier.fit_transform] X_train.shape=(6611, 18),X_test.shape=(3257, 18)
[ 2018-04-26 15:03:39,466][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(6611, 18), X_cur_test.shape=(3257, 18)
[ 2018-04-26 15:03:40,284][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=100.00%
[ 2018-04-26 15:03:41,333][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=100.00%
[ 2018-04-26 15:03:42,406][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0

('Test Accuracy of ExtraTrees = 99.969297 %', 0, 'fois')
(' Time ', '10.263', ' seconds')


[ 2018-04-26 15:03:53,059][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=100.00%
[ 2018-04-26 15:03:54,363][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=100.00%
[ 2018-04-26 15:03:55,448][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=100.00%
[ 2018-04-26 15:03:56,540][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=100.00%
[ 2018-04-26 15:03:57,628][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=100.00%
[ 2018-04-26 15:03:58,642][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=100.00%
[ 2018-04-26 15:03:59,702][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=100.00%
[ 2018-04-26 15:04:00,780][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=

('Test Accuracy of ExtraTrees = 99.969297 %', 1, 'fois')
(' Time ', '10.952', ' seconds')


[ 2018-04-26 15:04:06,372][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=100.00%
[ 2018-04-26 15:04:07,624][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.85%
[ 2018-04-26 15:04:08,728][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=100.00%
[ 2018-04-26 15:04:09,756][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=100.00%
[ 2018-04-26 15:04:10,784][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=100.00%
[ 2018-04-26 15:04:11,891][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=100.00%
[ 2018-04-26 15:04:12,978][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=100.00%
[ 2018-04-26 15:04:14,040][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=1

...

('Test Accuracy of ExtraTrees = 99.938594 %', 17, 'fois')
(' Time ', '11.054', ' seconds')


[ 2018-04-26 15:07:40,687][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.85%
[ 2018-04-26 15:07:41,755][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.85%
[ 2018-04-26 15:07:42,797][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=100.00%
[ 2018-04-26 15:07:43,826][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=100.00%
[ 2018-04-26 15:07:44,899][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=100.00%
[ 2018-04-26 15:07:46,017][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=100.00%
[ 2018-04-26 15:07:47,056][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=100.00%
[ 2018-04-26 15:07:48,240][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=99

('Test Accuracy of ExtraTrees = 99.938594 %', 18, 'fois')
(' Time ', '10.892', ' seconds')


[ 2018-04-26 15:07:54,106][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=100.00%
[ 2018-04-26 15:07:55,184][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=100.00%
[ 2018-04-26 15:07:56,263][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=100.00%
[ 2018-04-26 15:07:57,463][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=100.00%
[ 2018-04-26 15:07:58,473][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=100.00%
[ 2018-04-26 15:07:59,533][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=100.00%
[ 2018-04-26 15:08:00,630][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=100.00%
[ 2018-04-26 15:08:01,782][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=

('Test Accuracy of ExtraTrees = 99.938594 %', 19, 'fois')
(' Time ', '10.904', ' seconds')


# ('Test Accuracy of ExtraTrees = 99.969297 %', 0, 'fois')

# (' Time ', '10.263', ' seconds')